<font face="黑体" size=6>风格因子计算方法</font>

In [1]:
%pylab inline --no-import-all
from pathlib import Path
import pandas as pd
import numpy as np
from pandas import DataFrame
from pandas import Series
import statsmodels.api as sm

Populating the interactive namespace from numpy and matplotlib


/Users/davidyujun/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


# Size(市值因子)

　　定义：　　1.0 * LNCAP

　　LNCAP　　<span style="border-bottom:2px solid black">Natural log of market cap</span>

　　　　　　　Given by the logarithm of the total market capitalization of the firm.

# Beta(贝塔因子)

　　Definition:　　1.0 * BETA

　　BETA　　　　Beta($\beta$)

Computed as the slope coefficient in a time-series regression of excess stock return, $r_t-r_{ft}$, against the cap-weighted excess return of the estimation universe $R_t$,
$$r_t - r_{ft} = \alpha + \beta R_t+e_t\tag{1}$$
The regression coefficients are estimated over thr trailling 252 days of returns with a half-life of 63 trading days.

其中$r_ft$是无风险收益率日序列，$r_t$是股票收益率日序列，$R_t$是市值加权指数(如中证全指、万德全A指数)超额收益序列，回归系数采取过去252个交易日的收益数据，采用指数加权移动平均算法，半衰期为63个交易日（时间越接近权重越大）

按照普通最小二乘法，对于参数的估计为：
$$\beta=\frac{Cov(x,y)}{Var(x)}=\frac{\sum_{i=1}^n(x_i-\bar{x})(y_i-\bar{y})}{\sum_{i=1}^n(x_i-\bar{x})^2}$$

指数加权移动平均(Exponentially Weighted Moving Average, EWMA)，是BARRA model中常用的一种加权方式，按照时间远近呈指数衰减，按照指数加权移动平均，对于参数的估计为：
$$\beta=\frac{\sum_{i=0}^tw_i(x_i-\bar{x})(y_i-\bar{y})}{\sum_{i=0}^tw_i(x_i-\bar{x})^2}$$
其中，$t$是数据的时间长度减去1，这里为252，$x_t$是距离现金最近的数据，其权重为$w_0$。指数加权移动平均算法见[附录](#appendix)。
## 计算示例

In [40]:
secu_code = 'SH600036'
benchmark_code = 'SH000001'
calc_date = '2017-12-29'
days=252
# 取得个股复权行情数据
quote_header = ['code','date','open','high','low','close','vol','amount','turnover1','turnvoer2','factor']
quote_data_path = Path('/Volumes/DB/FactorDB/ElementaryFactor/mkt_daily_FQ/', '%s.csv' % secu_code)
df_secu_quote = pd.read_csv(quote_data_path,names=quote_header,header=0)
# 使用过去252个交易日的复权行情数据
df_secu_quote = df_secu_quote[df_secu_quote.date <= calc_date].tail(days+1)
df_secu_quote.reset_index(drop=True, inplace=True)
# 计算个股的日收益率序列
arr_close = np.array(df_secu_quote.iloc[1:]['close'])
arr_pre_close = np.array(df_secu_quote.shift(1).iloc[1:]['close'])
arr_secu_daily_ret = arr_close / arr_pre_close - 1.

# 取得基准复权行情数据
quote_data_path = Path('/Volumes/DB/FactorDB/ElementaryFactor/mkt_daily_FQ/', '%s.csv' % benchmark_code)
df_benchmark_quote = pd.read_csv(quote_data_path, names=quote_header,header=0)
df_benchmark_quote = df_benchmark_quote[df_benchmark_quote['date'].isin(list(df_secu_quote.date))]
df_benchmark_quote.reset_index(drop=True, inplace=True)
# 计算基准的日收益率序列
arr_close = np.array(df_benchmark_quote.iloc[1:]['close'])
arr_pre_close = np.array(df_benchmark_quote.shift(1).iloc[1:]['close'])
arr_benchmark_daily_ret = arr_close / arr_pre_close - 1.

# 计算权重 - 指数移动加权平均
T = len(arr_benchmark_dail_ret)
time_spans = sorted(range(T), reverse=True)
alpha = 1 - np.exp(np.log(0.5)/63)
x = [1-alpha] * T
y = [alpha] * (T - 1)
y.insert(0,1)
weights = np.float_power(x, time_spans) * y

# 采用加权最小二乘法计算beta
arr_benchmark_daily_ret = sm.add_constant(arr_benchmark_daily_ret)
cap_model = sm.WLS(arr_secu_daily_ret, arr_benchmark_daily_ret, weights=weights)
results = cap_model.fit()
results.params

array([0.00108207, 1.02427672])

# Momentum(动量因子)
## 定义
    Definintion:    1.0*RSTR

    RSTR            Relative strength
    
Computed as the sum of excess log returns over the trailing T = 504 trading days with a lag of L=21 trading days,
$$RSTR = \sum_{i=L}^{T+L}w_t[ln(1+r_t)-ln(r+r_{ft}],\tag{2}$$

where $r_t$ is the stock return on day t, $r_{ft}$ is the risk-free return, and $w_t$ is an exponential weight with a half-life of 126 trading days.
## 计算示例 

In [3]:
secu_code = 'SH600000'
calc_date = '2017-12-29'
days_start = 504
days_end = 21
half_life = 126

# 取得个股复权行情数据
quote_header = ['code','date','open','high','low','close','vol','amount','turnover1','turnvoer2','factor']
quote_data_path = Path('/Volumes/DB/FactorDB/ElementaryFactor/mkt_daily_FQ/', '%s.csv' % secu_code)
df_secu_quote = pd.read_csv(quote_data_path,names=quote_header,header=0)
# 使用过去第21个交易日到第504个交易日的复权行情数据
df_secu_quote = df_secu_quote[df_secu_quote.date <= calc_date].tail(days_start+1)
df_secu_quote = df_secu_quote.head(len(df_secu_quote)-days_end)
df_secu_quote.reset_index(drop=True, inplace=True)
# 计算个股的日对数收益率序列
arr_close = np.array(df_secu_quote.iloc[1:]['close'])
arr_pre_close = np.array(df_secu_quote.shift(1).iloc[1:]['close'])
arr_secu_daily_ret = np.log(arr_close / arr_pre_close)
# 计算权重 - 指数移动加权平均
T = len(arr_secu_daily_ret)
time_spans = sorted(range(T), reverse=True)
alpha = 1 - np.exp(np.log(0.5)/half_life)
x = [1-alpha] * T
y = [alpha] * (T - 1)
y.insert(0,1)
weights = np.float_power(x, time_spans) * y
# 计算RSTR
rstr = np.sum(arr_secu_daily_ret * weights)
rstr

-0.0004597495866620215

# Residual Volatility(残差波动率因子)
    Definition:    0.74*DASTD + 0.16*CMRA + 0.10*HSIGMA
## DASTD(Daily standard deviation)
### 定义
Computed as that the volatility of daily excess return over the past 252 trading days with a half-life of 42 trading days.

是过去252个交易日日超额收益率波动率，按照指数加权权重加权平均，半衰期为42个交易日。
$$DASTD = \sqrt{\sum_{t=1}^Tw_t(r_t-u(r))^2}\tag{3}$$
### 计算示例

In [4]:
secu_code = 'SH600000'
calc_date = '2017-12-29'
trailing = 252
half_life = 42

# 取得个股复权行情数据
quote_header = ['code','date','open','high','low','close','vol','amount','turnover1','turnvoer2','factor']
quote_data_path = Path('/Volumes/DB/FactorDB/ElementaryFactor/mkt_daily_FQ/', '%s.csv' % secu_code)
df_secu_quote = pd.read_csv(quote_data_path,names=quote_header,header=0)
# 使用过去第252个交易日的复权行情数据
df_secu_quote = df_secu_quote[df_secu_quote.date <= calc_date].tail(trailing+1)
df_secu_quote.reset_index(drop=True, inplace=True)
# 计算个股的日对数收益率序列及收益率均值
arr_close = np.array(df_secu_quote.iloc[1:]['close'])
arr_pre_close = np.array(df_secu_quote.shift(1).iloc[1:]['close'])
arr_secu_daily_ret = np.log(arr_close / arr_pre_close)
avg_daily_ret = np.mean(arr_secu_daily_ret)
# 计算权重 - 指数移动加权平均
T = len(arr_secu_daily_ret)
time_spans = sorted(range(T), reverse=True)
alpha = 1 - np.exp(np.log(0.5)/half_life)
x = [1-alpha] * T
y = [alpha] * (T - 1)
y.insert(0,1)
weights = np.float_power(x, time_spans) * y
# 计算DASTD
dastr = np.sqrt(np.sum((arr_secu_daily_ret - avg_daily_ret)**2 * weights))
print('DASTR factor of %s = %f' % (secu_code, dastr))

DASTR factor of SH600000 = 0.009623


## CMRA(Cumulative range)
### 定义
This descriptor differentiates stocks that have experienced wide swing ove the last 12 months from those that have traded within a narrow range. Let $Z(T)$ be the cumulative excess log return over the past $T$ months, with each month defined as the previous 21 trading days

$$Z(T) = \sum_{\tau=1}^T[ln(1+r_\tau) - ln(1+r_{f\tau})],\tag{4}$$

where $r_\tau$ is the stock return for month $\tau$(compounded over 21 days), and $r_{f\tau}$ is the risk-free return. The cumulative range is given by

$$CMRA = ln(1+Z_{max}) - ln(1+Z_{min}),\tag{5}$$

where $Z_{max}=max\{Z(T)\},Z_{min}=min\{Z(T)\}$, and $T=1,\dots,12$.

CMRA是过去12个月超额收益的离差，也是表征股票收益率的波动大小，$Z(T)$是过去$T$个月超额收益对数值的累计值，$Z(T)$是一个时间序列，$T=1,2,3,\dots,12$。

### 计算示例

In [17]:
secu_code = 'SZ002129'
calc_date = '2017-12-29'
trailing = 12
days_scale = 21

# 取得个股复权行情数据
quote_header = ['code','date','open','high','low','close','vol','amount','turnover1','turnvoer2','factor']
quote_data_path = Path('/Volumes/DB/FactorDB/ElementaryFactor/mkt_daily_FQ/', '%s.csv' % secu_code)
df_secu_quote = pd.read_csv(quote_data_path,names=quote_header,header=0)
# 使用过去第252个交易日的复权行情数据
df_secu_quote = df_secu_quote[df_secu_quote.date <= calc_date].tail(trailing*days_scale+1)
df_secu_quote.reset_index(drop=True, inplace=True)
# print(df_secu_quote)
# 计算个股的日对数收益率序列
# arr_close = np.array(df_secu_quote.iloc[1:]['close'])
# arr_pre_close = np.array(df_secu_quote.shift(1).iloc[1:]['close'])
# arr_secu_daily_ret = np.log(arr_close / arr_pre_close)
# print(arr_secu_daily_ret)
# 从期初开始每隔days_scale个交易日累加日对数收益率
# z = []
# for t in range(1, trailing+1):
#     k = t * days_scale -1
#     if k > len(arr_secu_daily_ret)-1:
#         k = len(arr_secu_daily_ret)-1
#         z.append(np.sum(arr_secu_daily_ret[:k]))
#         break
#     else:
#         z.append(np.sum(arr_secu_daily_ret[:k]))
        

# 计算每个月的个股变化率(1+r)
z = []
for t in range(1, trailing+1):
    k = t * days_scale
    if k > len(df_secu_quote)-1:
        k = len(df_secu_quote)-1
        z.append(df_secu_quote.iloc[k]['close']/df_secu_quote.iloc[0]['close'])
        break
    else:
        z.append(df_secu_quote.iloc[k]['close']/df_secu_quote.iloc[0]['close'])



print(z)
cmra = np.log(max(z)) - np.log(min(z))
print('CMRA factor of %s = %f' % (secu_code, cmra))

[1.2360516965704125, 0.6957086657544711, 0.9012882959961206, 0.43819772404129387, 0.5613729469166041, 0.6686691726304281, 0.5502145100861439, 0.386266412569766, 0.3459228669708005, 0.3716733434024893, 0.430489363038373, 0.4961211099133826]
CMRA factor of SZ002129 = 1.273462


# <a id='appendix'>附录：指数加权移动平均</a>
指数加权移动平均的权重有两种表现形式，一种是递推公式，如下：

$y_0 = x_0$

$y_t = (1-\alpha)y_{t-1} + \alpha x_t$

根据递推公式，可以得到

$y_t = (1-\alpha)^tx_0 + (1-\alpha)^{t-1}\alpha x_1 + \ldots + (1-\alpha)\alpha x_{t-1} + \alpha x_t$

权重可以写为，

$
\left\{
\begin{array}{ll}
w_i = (1-\alpha)^i\alpha & (i < t)\\
w_i = (1-\alpha)^i       & (i = t)
\end{array}
\right.
$

可以得到$\sum_{i=0}^t = 1$.注意$i$表示距离现在时间，$i$越大，距离现在时间越长。

在python中可以直接调用pandas.ewma()或者pandas.ewm().mean()实现。注意的是，默认参数为adjust=True，采用近似的权重，权重为$w_i=(1-\alpha)^i$，当参数adjust=False，才会采用以上精确推算结果。

半衰期$h$和参数$\alpha$的关系为$\alpha = 1 - e^{\frac{ln(0.5)}{h}}$，这是因为最大权重$w_0 = \alpha$，经过了$h$天之后权重为$w_h = (1-\alpha)^h\alpha$，两者之比$\frac{w_h}{w_0}=(1-\alpha)^h=0.5$，可以得出$\alpha = 1 - e^{\frac{ln(0.5)}{h}}$，也就是说经过了$h$天后，权重变成了初始权重的一半。